<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#Primeiro-Passo:-Armazenamento" data-toc-modified-id="Primeiro-Passo:-Armazenamento-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Primeiro Passo: Armazenamento</a></span><ul class="toc-item"><li><span><a href="#Virtual-Private-Cloud" data-toc-modified-id="Virtual-Private-Cloud-1.1"><span class="toc-item-num">1.1&nbsp;&nbsp;</span>Virtual Private Cloud</a></span></li></ul></li></ul></div>

Entre algumas evoluções obtidas em um rico processo de construção e documentação do projeto _NBAFlow_, este notebook possui, como ponta de partida, o resultado obtido a partir da primeira entrega do [Painel de Estatísticas de Jogadores](https://public.tableau.com/app/profile/thiago.henrique.gomes.panini/viz/NBAFlow-InsightsdeDadosdaNBA/PaineldeEstatsticasdeJogadores) já publicado no [Tableau Public](https://public.tableau.com/s/). Dentro deste primeiro produto entregue, é possível consolidar o fluxo dos dados a partir do seguinte diagrama:

<div align="center">
    <br><img src="https://i.imgur.com/HMfAMMV.jpg" alt="nbaflow-c4">
</div>

Em resumo, o painel analítico construído e publicado no Tableau é gerado a partir de uma fonte de dados, concebida em formato csv e armazenada localmente, cuja construção tem, como base, o processamento de scripts Python capazes de consumir _endpoints_ presentes na biblioteca `nba_api`. Neste primeiro momento, a solução adotada atendia perfeitamente os requisitos estabelecidos, permitindo assim uma rápida construção de um produto relevante.

Em sequência ao desenvolvimento inicial proposto, o objetivo deste notebook é dar início aos passos necessários a serem concebidos visando um aprimoramento na arquitetura existente, abrindo margem para armazenamento e processamento dos dados em nuvem através da [AWS](https://aws.amazon.com/pt/). Conceitos e serviços como VPC, subredes, RDS, EC2, S3, SNS, entre outros, podem eventualmente serem abordados neste material. O intuito é explorar algumas possibilidades previamente estabelecidas e, ao longo do desenvolvimento, mapear novas ideias que possam surgir.

___
**_Para entender todo o contexto do projeto, é sugerido o consumo dos seguintes materiais:_**

* [1. NBAFlow - documentação e exploração inicial da API nba_api](https://github.com/ThiagoPanini/nbaflow/blob/main/notebooks/NBAflow-doc-api.ipynb)

* [2. NBAFlow - documentação de fluxo de extração de imagens de jogadores](https://github.com/ThiagoPanini/nbaflow/blob/main/notebooks/NBAflow-players-images.ipynb)

* [3. NBAFlow - estruturação de armazenamento em banco de dados Postgres (OPCIONAL)](https://github.com/ThiagoPanini/nbaflow/blob/main/notebooks/NBAflow-psql.ipynb)

* [4. Classes estruturadas de processamento de dados](https://github.com/ThiagoPanini/nbaflow/tree/main/core)

* [5. Scripts de processamento de dados utilizando as classes estruturadas](https://github.com/ThiagoPanini/nbaflow/tree/main/scripts)
___

# Primeiro Passo: Armazenamento

Como mencionado anteriormente, a primeira solução do _dashboard analítico_ do projeto NBAFlow tem, como fonte de dados, um arquivo `.csv` gerado a partir da execução de scripts Python consturídos justamentepara este fim. Como uma primeira tentativa de contato com serviços da AWS, é possível analisar as possibilidades de armazenar estes dados em um serviço de banco de dados na nuvem. Dentro da construção do arquivo e, considerando seu conteúdo, o serviço da AWS que mais se enquadra neste cenário é o banco de dados relacional [RDS](https://aws.amazon.com/pt/rds/) (do inglês, _Relational Database Service_). Da página oficial do serviço, temos:

> _O Amazon Relational Database Service (Amazon RDS) facilita a configuração, a operação e a escalabilidade de bancos de dados relacionais na nuvem. O serviço oferece capacidade econômica e redimensionável e automatiza tarefas demoradas de administração, como provisionamento de hardware, configuração de bancos de dados, aplicação de patches e backups. Dessa forma, você pode se concentrar na performance rápida, alta disponibilidade, segurança e conformidade que os aplicativos precisam._
_O Amazon RDS está disponível em vários tipos de instância de banco de dados – com otimização para memória, performance ou E/S – e oferece seis mecanismos de bancos de dados comuns, incluindo Amazon Aurora, PostgreSQL, MySQL, MariaDB, Oracle Database e SQL Server. Você pode usar o AWS Database Migration Service para migrar ou replicar facilmente bancos de dados existentes para o Amazon RDS._

Como uma forma estrutural de propor a construção e a implantação de serviços da AWS em um ambiente isolado capaz de fornecer toda a suite de conectividade exigida pelo projeto, os próximos passos dessa seção de armazenamento serão pautados pela organização de redes e grupos de acessos pertinentes à proposta em questão. Assim, criações de novas [VPCs](https://aws.amazon.com/pt/vpc/) (do inglês, _Virtual Private Cloud_), sub-redes e grupos de segurança serão exemplificados como forma de garantir a estrutura do projeto

## Virtual Private Cloud

Da página do serviço, disponibilizada pela própria AWS, temos:

>_A Amazon Virtual Private Cloud (Amazon VPC) é um serviço que permite iniciar recursos da AWS em uma rede virtual isolada logicamente definida por você. Você tem controle total sobre seu ambiente de redes virtuais, incluindo a seleção do seu próprio intervalo de endereços IP, a criação de sub-redes e a configuração de tabelas de rotas e gateways de rede. Você pode usar IPv4 e IPv6 para a maioria dos recursos em sua nuvem privada virtual, garantindo acesso fácil e seguro a recursos e aplicações._
_Como um dos serviços básicos da AWS, o Amazon VPC facilita a personalização da configuração de rede da VPC. Você pode criar uma sub-rede voltada ao público para seus servidores Web que têm acesso à Internet. Também é possível colocar seus sistemas back-end, como bancos de dados ou servidores de aplicações, em uma sub-rede privada, sem acesso à Internet. Com o Amazon VPC, você pode usar várias camadas de segurança, incluindo grupos de segurança e listas de controle de acesso à rede, para ajudar a controlar o acesso às instâncias do Amazon EC2 em cada sub-rede._

Assim, dentro do objetivo simplório em criar um banco de dados relacional na nuvem capaz de armazenar os dados de jogadores da NBA obtidos a partir da biblioteca `nba_api`, criar uma rede isolada de recursos da provedora cloud é um passo importante para a implantação de recursos e a manutenção da organização de acessos entre os serviços e a internet pública.

Por padrão, a AWS fornece uma VPC já com um bloco [CIDR](https://en.wikipedia.org/wiki/Classless_Inter-Domain_Routing) (do inglês, _Classeless Inter-Domain Route)_ associado. No exemplo abaixo, a VPC padrão `vpc-ea4ab68c` pode ser vista em detalhes, incluindo um parâmetro extremamente importante dentro do entendimento da conectividade interna e externa dos dispositivos internos dessa rede: o bloco de IPs CIDR. Na imagem, a partir do endereço `172.31.0.0/16`, associado ao bloco CIDR da VPC, é possível extrair, por exemplo, informações como:

* Trata-se de um endereço de IP da classe B
    - Primeiro octeto iniciado por 172 está entre 128 e 191, determinando assim o pertencimento à classe B
    - Por ser de classe B, existem $2^{16}=65536$ possibilidades de conexão (redes, hosts e broadcasts)
    - Máscara de sub-rede equivalente a 255.255.0.0 (característica de IPs da classe B)
* A notação CIDR indica que existem diferentes sub-redes 
    - O número 16, após a barra "/", indica que são utilizados 16 bits "1s" na máscara de sub-rede atrelada
    - Transformando em binário, temos que a máscara equivale aos octetos: 11111111.11111111.00000000.00000000
    - Após essa confirmação, de fato, a máscara de sub-rede equivale a 255.255.0.0, mantendo assim as 65536 possibilidades de conexão

<div align="center">
    <br><img src="https://i.imgur.com/dCobYeO.png" alt="aws-defaultvpc">
</div>

Após esse entendimento inicial, será proposta a criação de uma nova VPC para alocação dos recursos de novos projetos dentro da AWS. Considerando um cenário mais enxuto em termos de possibilidades de conexão, a nova VPC criada terá, como proposta, uma alocação reduzida de conexões a partir de um bloco CIDR com uma abertura menor de redes e hosts. Assim, a figura abaixo ilustra a `vpc-project-development` criada para o fim especificado:

<div align="center">
    <br><img src="https://i.imgur.com/xSvHIBS.png" alt="aws-defaultvpc">
</div>

Ao observar a imagem, é possível notar o bloco CIDR de IP defindo por `192.168.1.0/24`, indicando assim um intervalo de IPs representado pela máscara de sub-rede definida por `11111111.11111111.11111111.00000000` (ou `255.255.255.0`, em notação decimal)